In [3]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [4]:
#Argentina's banking system data (source: BCRA)
data=pd.read_excel('BankingSystemDataArgentina2018.xlsx',index_col='Bank')
matrix=pd.read_excel('Matrix_2018.xlsx',index_col=0)

## Debt Rank (Battiston et al. 2012)
Implementation of Battiston's debt rank for determining SIFIs in a given banking system. 

Paper: https://www.nature.com/articles/srep00541

In [5]:
class DebtRank:
    
    def __init__(self,df,matrix):
        self.df=df
        self.matrix=matrix
    
    def init_losses(self,bank):
        exposure=self.matrix[bank].copy()
        impact=self.df['Buffer']-exposure
        defaulted=list(impact[impact<=0].index)
        n_defaults=len(defaulted)
        exposure[impact<0]=0

        bank_loss=self.df['Buffer'].loc[bank]
        dir_loss=sum((impact<0)*self.df['Buffer'])+sum(exposure)

        return {'Bank':bank,'Bank loss':bank_loss,'Direct loss':dir_loss,'Defaults':n_defaults}, defaulted
    
    def total_losses(self,bank):
        dic,defaulted=self.init_losses(bank)
        dic['Indirect loss']=0
        
        while defaulted!=[]:
            for i in defaulted:
                a,b=self.init_losses(i)
                dic['Indirect loss']+=a['Direct loss']
                dic['Defaults']+=a['Defaults']
                defaulted.remove(i)
                defaulted.extend(b)
                
        return dic 
    
    def system(self):
        banks=list(self.df.index)
        l=[]
        for j in banks:
            l.append(self.total_losses(j))
            
        summary=pd.DataFrame(l)
        summary.index=summary['Bank']
        summary.drop('Bank',axis=1,inplace=True)
        summary['Damage']=summary['Direct loss']+summary['Indirect loss'] 
        
        return summary.sort_values(by='Damage',ascending=False)

In [10]:
s=DebtRank(data,matrix)
s.system()

,Bank loss,Direct loss,Defaults,Indirect loss,Damage
Bank,,,,,
Ciudad,1625.742563,3903.854437,2,2.7,3906.554437
Nacion,13718.044625,1081.780625,1,0.0,1081.780625
Provincia,3971.575625,301.780625,1,0.0,301.780625
Santander,2712.349000,192.200000,0,0.0,192.200000
Galicia,3924.994360,165.400000,0,0.0,165.400000
BBVA,4591.733030,154.400000,0,0.0,154.400000
Ita˙,781.233500,122.200000,0,0.0,122.200000
Patagonia,1922.140125,98.900000,0,0.0,98.900000
Macro,11224.205370,93.200000,0,0.0,93.200000
